In [1]:
import numpy as np

In [2]:
# Read Key
key_array = np.zeros(shape=(4,4)).astype(np.str_)
f = open('aes_sample.in','rb')
n = 0;
i = 0
j = 0
s = f.read(1)
while s:
    byte = ord(s)
    if i == 3 and j == 3:
        key_array[i][j] = hex(byte)
        break
    if i < 3:
        key_array[i][j] = hex(byte)
        i += 1
    else:
        if j < 3:
            key_array[i][j] = hex(byte)
            i = 0
            j += 1
    n = n+1
    # print('0x%02x,'%(byte),end='')
    if n%16==0:
            print('')
    s = f.read(1)
f.close()
key_array

array([['0xf4', '0xa1', '0x34', '0x7e'],
       ['0xc0', '0xf6', '0x3f', '0x6a'],
       ['0x20', '0x4', '0xac', '0xe0'],
       ['0xa0', '0xfd', '0x6a', '0xf9']], dtype='<U32')

In [3]:
# Read plaintext (test)
plaintext_array = np.zeros(shape=(4,4)).astype(np.str_)
f = open('aes_sample.in','rb')
block_counter = 0
n = 0;
i = 0
j = 0
s = f.read(1)
while s:
    if block_counter == 0:
        n = n+1
        if n%16==0 and n != 0:
             block_counter += 1
        s = f.read(1)
        continue
    byte = ord(s)
    if i == 3 and j == 3:
        plaintext_array[i][j] = hex(byte)
        break
    if i < 3:
        plaintext_array[i][j] = hex(byte)
        i += 1
    else:
        if j < 3:
            plaintext_array[i][j] = hex(byte)
            i = 0
            j += 1
    n = n+1
    s = f.read(1)
f.close()
plaintext_array

array([['0xf2', '0x8b', '0xd9', '0xe4'],
       ['0x95', '0x99', '0x3d', '0x49'],
       ['0xb9', '0x44', '0x98', '0xaf'],
       ['0x31', '0x34', '0xa4', '0xd8']], dtype='<U32')

In [4]:
f = open('aes_sample.in','rb')
n = 0;
s = f.read(1)
while s:
    byte = ord(s)
    n = n+1
    print('0x%02x,'%(byte),end='')
    if n%16==0:
            print('')
    s = f.read(1)
print('\n\ntotal bytes: %d'%n)
f.close()

0xf4,0xc0,0x20,0xa0,0xa1,0xf6,0x04,0xfd,0x34,0x3f,0xac,0x6a,0x7e,0x6a,0xe0,0xf9,
0xf2,0x95,0xb9,0x31,0x8b,0x99,0x44,0x34,0xd9,0x3d,0x98,0xa4,0xe4,0x49,0xaf,0xd8,


total bytes: 32


In [93]:
#列混合 (MixColumns)
def xtime(char, number):
    char_int = int(char,16)
    if number == 1:
        return char_int
    tmp = (char_int << 1) & 0xff #
    if number == 2:
        return tmp if char_int < 128 else tmp ^ 0x1b
    if number == 3:
        return xtime(char, 2) ^ char_int
        
def mixcolumns_culculate(p1, p2, p3, p4, n):
    if n == 0:
        return hex(xtime(p1, 2) ^ xtime(p2, 3) ^ xtime(p3, 1) ^ xtime(p4, 1))
    elif n == 1:
        return hex(xtime(p1, 1) ^ xtime(p2, 2) ^ xtime(p3, 3) ^ xtime(p4, 1))
    elif n == 2:
        return hex(xtime(p1, 1) ^ xtime(p2, 1) ^ xtime(p3, 2) ^ xtime(p4, 3))
    else:
        return hex(xtime(p1, 3) ^ xtime(p2, 1) ^ xtime(p3, 1) ^ xtime(p4, 2))

def mixcolumns(array):
    result_array = np.zeros(shape=(4,4)).astype(np.str_)
    for i in range(4):
        for j in range(4):
            result_array[j][i] = mixcolumns_culculate(array[0][i], array[1][i], array[2][i], array[3][i], j)
    return result_array



In [8]:
test_array = np.zeros(shape=(4,4)).astype(np.str_)
test_array[0][0] = "0xd4"
test_array[0][1] = "0xe0"
test_array[0][2] = "0xb8"
test_array[0][3] = "0x1e"
test_array[1][0] = "0xbf"
test_array[1][1] = "0xb4"
test_array[1][2] = "0x41"
test_array[1][3] = "0x27"
test_array[2][0] = "0x5d"
test_array[2][1] = "0x52"
test_array[2][2] = "0x11"
test_array[2][3] = "0x98"
test_array[3][0] = "0x30"
test_array[3][1] = "0xae"
test_array[3][2] = "0xf1"
test_array[3][3] = "0xe5"
test_array

array([['0xd4', '0xe0', '0xb8', '0x1e'],
       ['0xbf', '0xb4', '0x41', '0x27'],
       ['0x5d', '0x52', '0x11', '0x98'],
       ['0x30', '0xae', '0xf1', '0xe5']], dtype='<U32')

In [94]:

mixcolumns(test_array)

array([['0x4', '0xe0', '0x48', '0x28'],
       ['0x66', '0xcb', '0xf8', '0x6'],
       ['0x81', '0x19', '0xd3', '0x26'],
       ['0xe5', '0x9a', '0x7a', '0x4c']], dtype='<U32')